<a href="https://colab.research.google.com/github/Roarou/Computational-control/blob/main/COCO_T6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational control - Tutorial 6

We illustrate how to identify a linear time-invariant system via its impulse response using the Kalman-Ho algorithm. 

### Package imports

In [1]:
import numpy  as np                        # For linear algebra
import matplotlib.pyplot as plt            # For plots
np.random.seed(1)                          # Generate random seed
np.set_printoptions(precision=1)           # Set nice printing format

### Problem setup

Consider the linear time-invariant system described by the equations

$$
\begin{align}
x_{t+1} &= A x_t + B u_t\\
y_{t}   &= C x_t + D u_t
\end{align}
$$

with $x_t \in \mathbb{R}^2$, $u_t \in \mathbb{R}^2$, $y_t \in \mathbb{R}^2$  and constant matrices $A \in \mathbb{R}^{2\times 2}$, $B \in \mathbb{R}^{2\times 2}$, $C \in \mathbb{R}^{2\times 2}$, and $D \in \mathbb{R}^{2\times 2}$ defined as

$$
\begin{align}
A = 
\begin{bmatrix}
1 & 0 \\
0 & 2
\end{bmatrix}, \quad
B =  I , \quad
C =  I, \quad
D = 0 .
\end{align}
$$


In [2]:
# System parameters   
A = np.array([[1.0, 0.0],                  # A - State-space matrix                      
              [0.0, 2.0]])                 # 
B = np.eye(2)                              # B - State-space matrix                      
C = np.eye(2)                              # C - State-space matrix 
D = np.zeros([2,2])                        # D - State-space matrix 
                                  
# Integer invariants
n = A.shape[0]                             # n = number of states
m = B.shape[1]                             # m = number of inputs
p = C.shape[0]                             # p = number of output

### Data collection

The Kalman-Ho algorithm requires a sequence of _Markov parameters_

$$
 G_t = 
\left \{
\begin{array}[rcl] 
\, D &\text{ for } t=0\\ 
CA^{t-1}B &\text{ for } t=1, 2, \ldots, T
\end{array}
\right.
$$

to determine the system matrices. 


The sequence can be obtained directly from the response of the system to a _unit impulse_ of length $T$, defined as 

$$
u_t = 
\left \{
\begin{array}[rcl] 
\, I &\text{ for } t=0\\ 
\mathbb{0} &\text{ for } t=1, 2, \ldots, T
\end{array}
\right.
$$

starting from zero initial conditions. In the following script, we select $T=8.$



In [10]:
# Data collection parameters
T = 8                                            # Length of unit impulse 
impulse = [np.eye(m)]+[np.zeros((m,m))]*T  # Unit impulse (of length T+1)                                           
x = np.zeros((n,n))                              # Initial condition (zero)

# Data collection
impulse_response = []                            # Markov parameters                        

##################################################
#           Implement the following              #
# 1. Collect data from the system using "impulse"#
# 2. Estimate the D term                         #
# 3. Define a list of Markov parameters          #
# 3. Define a list of shifted Markov parameters  #  
##################################################

# Data collection
impulse_response = []                            # Markov parameters                        
for t in range(T+1): 
    u = impulse[t]                               # Input
    y = C@x+D@u                                  # Output
    x = A@x+B@u                                  # State update  
    impulse_response.append(y)                   # Save output

D_estimate = impulse_response.pop(0)

markov_parameters = impulse_response[:-1]

markov_parameters_shifted = impulse_response[1:]

# Print results
print("Markov parameters:\n")                    # Print Markov parameters
print(*impulse_response, sep='\n')               #  

Markov parameters:

[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 2.]]
[[1. 0.]
 [0. 4.]]
[[1. 0.]
 [0. 8.]]
[[ 1.  0.]
 [ 0. 16.]]
[[ 1.  0.]
 [ 0. 32.]]
[[ 1.  0.]
 [ 0. 64.]]
[[  1.   0.]
 [  0. 128.]]


### Hankel matrices

The Kalman-Ho algorithm also requires a Hankel matrix and a shifted Hankel matrix defined using the sequence of Markov parameters.


The _Hankel matrix_ is defined as 

$$
{H}_{k,\ell}
=
\begin{bmatrix}
G_{1}   & G_{2} & \cdots & G_{\ell} \\
G_{2}   & G_{3} & \cdots & G_{\ell+1} \\
\vdots  & \vdots  & \ddots & \vdots    \\
G_{k}   & G_{k+1} & \cdots & G_{k+\ell-1} 
\end{bmatrix}
$$

and the _shifted Hankel matrix_ is defined as 

$$
{H}_{k,\ell}^{\uparrow}
=
\begin{bmatrix}
G_{2}   & G_{3} & \cdots & G_{\ell+1} \\
G_{3}   & G_{4} & \cdots & G_{\ell+2} \\
\vdots  & \vdots  & \ddots & \vdots    \\
G_{k+1}   & G_{k+2} & \cdots & G_{k+\ell} 
\end{bmatrix}
$$

where $k$ and $\ell$ are positive integers such that $k+\ell\le T.$

In this script, we select $k=3$ and $\ell=5$

In [11]:
# Hankel matrices - parameters
k = 3                                            # Number of block rows
l = 5                                            # Number of block columns

# Hankel matrix 
if (k+l<T+1):
    Hankel = np.empty((p*k,m*l)) 
    for i in range(k):
      for j in range(l):
        Hankel[i*p:(i+1)*p,
               j*m:(j+1)*m] = \
                markov_parameters[i+j]   
else:
    print("Need more data")                      # Need more data
    
print("Hankel = \n",Hankel)                      # Print Markov parameters      

# Shifted Hankel matrix
if (k+l<T+1):
    Hankel_shifted = np.empty((p*k,m*l)) 
    for i in range(k):
      for j in range(l):
        Hankel_shifted[i*p:(i+1)*p,
                 j*m:(j+1)*m] = \
                 markov_parameters_shifted[i+j]  # Define shifted Hankel    

# Shifted Hankel matrix
if (k+l<T+1):
    Hankel_shifted = np.empty((p*k,m*l)) 
##################################################
#     Construct the shifted Hankel matrix here   #   
##################################################

print("Hankel_shifted = \n",Hankel_shifted)      # Print Markov parameters      

Hankel = 
 [[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  2.  0.  4.  0.  8.  0. 16.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  2.  0.  4.  0.  8.  0. 16.  0. 32.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  4.  0.  8.  0. 16.  0. 32.  0. 64.]]
Hankel_shifted = 
 [[1.7e-316 0.0e+000 1.3e-316 6.9e-310 1.9e-152 9.6e+199 6.0e-154 9.9e+247
  4.5e-091 2.4e-154]
 [9.1e+276 9.8e+199 4.9e+252 2.3e+161 9.1e+223 3.0e-152 9.1e+242 6.0e-154
  8.4e+252 1.9e-153]
 [4.4e+252 9.8e+199 6.0e-154 6.4e+029 8.4e+252 2.2e+068 6.0e-154 2.4e-154
  1.1e+155 8.4e+252]
 [9.8e-153 5.3e+180 3.2e+180 4.8e+180 1.7e+243 2.7e-260 7.7e+218 6.0e-154
  4.5e-091 7.0e-009]
 [3.7e+233 1.4e+131 8.5e-096 6.0e-154 2.4e+232 9.1e+223 6.0e-154 6.0e-154
  1.0e-113 2.0e-062]
 [2.4e-154 7.1e+159 9.8e+199 2.0e-062 1.9e-009 6.0e-154 1.4e+131 9.0e-096
  6.0e-154 1.6e-322]]


### State space dimension

The Kalman-Ho algorithm requires an estimate of the dimension of the system. 

This can be retrieved directly from the rank of the Hankel matrix.

In [12]:
# Estimate state space dimension
N = np.linalg.matrix_rank(Hankel)
print("rank(Hankel) = ",N,"\n")

rank(Hankel) =  2 



### Kalman-Ho algorithm

We now have all the main ingredients to implement the Kalman-Ho algorithm.

Recall that the algorithm involves the following steps:

- Computing a thin SVD of the Hankel matrix
 $${H}_{k,\ell} = U_1 \Sigma_1 V_1^{\mathsf{T}}.$$
- Defining the observability matrix
 $$\mathbf{O}_k = U_1 \Sigma_1^{1/2} $$
 and the controllability matrix
 $$\mathbf{C}_\ell = \Sigma_1^{1/2} V_1^{\mathsf{T}}.$$
- Computing the system matrices as
$$
\begin{align}
A &= \mathbf{O}_k^{\dagger}{H}_{k,\ell}^{\uparrow}\mathbf{C}_\ell^{\dagger} \\
B &= \text{first } m \text{ columns of } \mathbf{C}_\ell \\
C &= \text{first } p \text{ rows of } \mathbf{O}_k \\
D &= G_0 \\
\end{align}
$$

where $M^{\dagger}$ is the Moore-Penrose inverse of $M$.








In [13]:
## Kalman-Ho algorithm

# Compute SVD (rounding to zero)
U = np.linalg.svd(Hankel)[0].round(decimals=12)[:,0:N]
Sigma = np.linalg.svd(Hankel)[1].round(decimals=12)[0:N]
V_T = np.linalg.svd(Hankel)[2].round(decimals=12)[0:N,:]

# Compute observability and controllability matrices
observability = U@np.diag(np.sqrt(Sigma))
controllability = np.diag(np.sqrt(Sigma))@V_T

# Compute A,B,C matrices
A_estimate = (np.linalg.pinv(observability)@ Hankel_shifted @ np.linalg.pinv(controllability)).round(decimals=12)
B_estimate = controllability[:,0:m]
C_estimate = observability[0:p,:]



In [14]:
print("A realization of the system is:\n")
print("A = \n",A_estimate,"\n")            # A - estimate
print("B = \n",B_estimate,"\n")            # B - estimate
print("C = \n",C_estimate,"\n")            # C
print("D = \n",D_estimate,"\n")       

A realization of the system is:

A = 
 [[ 1.4e+196 -4.9e+274]
 [-2.3e+251 -1.7e+260]] 

B = 
 [[ 0.   0.5]
 [-0.9  0. ]] 

C = 
 [[ 0.  -1.1]
 [ 2.   0. ]] 

D = 
 [[0. 0.]
 [0. 0.]] 

